In [ ]:
# Problems Notebook

In [ ]:
## Problem 1: Binary Words and Operations

### Introduction

SHA-256 (Secure Hash Algorithm 256-bit) is a cryptographic hash function that is part of the SHA-2 family, standardized by NIST in the [Federal Information Processing Standards Publication 180-4](link). The SHA-256 algorithm operates on 32-bit words (binary strings of length 32) and uses several bitwise logical functions and rotation operations to process input data.

In [ ]:
In this problem, we implement seven key functions defined on page 10 of the Secure Hash Standard:

**Logical Functions:**
- **Parity(x, y, z)** - Returns the bitwise XOR of three inputs
- **Ch(x, y, z)** - "Choose" function: uses x to choose bits from y or z
- **Maj(x, y, z)** - "Majority" function: returns the majority bit value at each position

**Rotation Functions:**
- **Σ₀²⁵⁶(x)** - Sigma0: combines three right rotations of x
- **Σ₁²⁵⁶(x)** - Sigma1: combines three right rotations of x  
- **σ₀²⁵⁶(x)** - sigma0: combines rotations and shifts for message schedule
- **σ₁²⁵⁶(x)** - sigma1: combines rotations and shifts for message schedule

All operations are performed on 32-bit unsigned integers, and we use NumPy's `uint32` data type to ensure proper handling of overflow and bitwise operations.

In [ ]:
### Implementation